# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh your knowledge of SQL Queries!

## Objectives

You will be able to:

- Use `GROUP BY` statements in SQL to apply aggregate functions like: `COUNT`, `MAX`, `MIN`, and `SUM`
- Decide and perform whichever type of join is best for retrieving desired data
- Use the `HAVING` clause to compare different aggregates
- Write subqueries to decompose complex queries

## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as Pandas. By combining them, we'll be able to write queries as Python strings, and make sure that the results are always returned as a Pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3` 
* Establish a connection to the database `data.sqlite` 
* Get the `cursor` from the connection and store it in the variable `c` 

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Basic Queries

Now, let's review basic SQL queries. In the cell below:

Write a query that gets the first name, last name, phone number, address, and credit limit for all customers in California with a credit limit greater than 25000.00. 

In [8]:
# For the first query, the boilerplate for getting 
# the query into a DataFrame has been provided for you
cur.execute("""
SELECT contactFirstName, contactLastName, phone, addressLine1, creditLimit
FROM customers
WHERE state == 'CA'
GROUP BY creditLimit
HAVING creditLimit > 25000.00
ORDER BY creditLimit DESC;
""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,contactFirstName,contactLastName,phone,addressLine1,creditLimit
0,Susan,Nelson,4155551450,5677 Strong St.,210500
1,Valarie,Thompson,7605558146,361 Furth Circle,105000
2,Julie,Young,6265557265,78934 Hillside Dr.,90700
3,Juri,Hashimoto,6505556809,9408 Furth Circle,84600
4,Sue,Frick,4085553659,3086 Ingle Ln.,77600


#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that gets the average credit limit per state.

In [9]:
cur.execute("""
SELECT state, AVG(creditLIMIT) AS avg_creditLimit
FROM customers
GROUP BY state
ORDER BY creditLimit DESC
""")
df2 = pd.DataFrame(cur.fetchall())
df2.columns = [x[0] for x in cur.description]
df2.head()

,state,avg_creditLimit
0,CA,83854.545455
1,Victoria,88800.000000
2,NY,89966.666667
3,NH,114200.000000
4,NSW,100550.000000


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [14]:
cur.execute("""
SELECT customers.customerName, customerNumber, orders.orderNumber, orders.status, orderdetails.quantityOrdered
FROM customers
JOIN orders
USING(customerNumber)
JOIN orderdetails
USING(orderNumber);
""")
df3 = pd.DataFrame(cur.fetchall())
df3.columns = [x[0] for x in cur.description]
df3.head()

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,Atelier graphique,103,10123,Shipped,26
1,Atelier graphique,103,10123,Shipped,34
2,Atelier graphique,103,10123,Shipped,46
3,Atelier graphique,103,10123,Shipped,50
4,Atelier graphique,103,10298,Shipped,32


#### Expected Output

<img src='images/joins.png' width='500' height='600'> 

## HAVING and ORDER BY

Now, return the customerName, customerNumber, productName, productCode, and total number ordered for any product a customer has bought 10 or more of cumulatively. Sort the rows in descending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY -- make sure you get the order of them correct!

In [39]:
cur.execute("""
SELECT customers.customerName, customers.customerNumber, products.productName, products.productCode,
SUM(orderdetails.quantityOrdered) AS total_num_orders
FROM customers JOIN orders USING(customerNumber)
JOIN orderdetails USING(orderNumber)
JOIN products USING (productCode)
GROUP BY customers.customerName, products.productCode
HAVING total_num_orders >= 10
ORDER BY total_num_orders DESC;
""")
df4 = pd.DataFrame(cur.fetchall())
df4.columns = [x[0] for x in cur.description]
df4.head()

,customerName,customerNumber,productName,productCode,total_num_orders
0,Euro+ Shopping Channel,141,1992 Ferrari 360 Spider red,S18_3232,308
1,Euro+ Shopping Channel,141,1958 Chevy Corvette Limited Edition,S24_2840,245
2,Euro+ Shopping Channel,141,1970 Dodge Coronet,S24_1444,197
3,Euro+ Shopping Channel,141,1957 Chevy Pickup,S12_4473,183
4,Euro+ Shopping Channel,141,2002 Chevy Corvette,S24_3432,174


#### Expected Output

<img src='images/having_order.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from offices with less than 5 employees. Print the first five rows of this DataFrame. 

In [47]:
cur.execute("""SELECT employeeNumber, firstName, lastName, officeCode
FROM employees 
WHERE officeCode In (SELECT officeCode
                    FROM offices
                    JOIN employees
                    USING(officeCode)
                    GROUP BY officeCode
                    HAVING COUNT(employeeNumber) <5);""")
df5 = pd.DataFrame(cur.fetchall())
df5.columns = [x[0] for x in cur.description]
df5.head()

,employeeNumber,firstName,lastName,officeCode
0,1088,William,Patterson,6
1,1188,Julie,Firrelli,2
2,1216,Steve,Patterson,2
3,1286,Foon Yue,Tseng,3
4,1323,George,Vanauf,3


#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!